<a href="https://colab.research.google.com/github/tasinfrancesco/soc_sci_code/blob/main/PDF_read_iterator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import glob
docs = glob.glob("SocSciPDFs/document_*")
for doc in docs:
  os.remove(doc)

In [26]:
# !pip install playwright
# !pip install nest_asyncio
# !playwright install
# !playwright install-deps
import urllib.request
import os
import time
import random
import requests
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
nest_asyncio.apply()
async def async_download_pdfs(pdf_url, filename, filenum):
      async with async_playwright() as p:
            #We define the browser, the context and the page
            browser = await p.firefox.launch(headless=True)
            context = await browser.new_context()
            page = await browser.new_page()
            async with page.expect_download() as download_info:
               #If we navigate without try, it will throw an exception and it will stop our script, so, we wrap it inside a try except block
              try:
                await page.goto(pdf_url)

              except:
                await print(f"I tried the hard way, but it still failed to download {pdf_url}: HTTP Error 403")
              download = await download_info.value
              await download.save_as(filename)
              context.close()
              browser.close()
              return await 1, filepath

def download_pdfs(pdf_url, output_dir, filenum):
    # Track successful and failed downloads. THIS FUNCTION WAS JUST TAKEN FROM STACK OVERFLOW!!!
    successful = 0
    failed = 0
    error_403 = 0

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a filename from the URL
    filename = os.path.join(output_dir, f"document_{filenum}.pdf")

    # If URL has a filename, use that instead

    try:
        # Download with stream=True for larger files
        response = requests.get(pdf_url, stream=True)

        if response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            print(f"Downloaded: {filename}")

        elif response.status_code == 403:
          loop = asyncio.get_event_loop()
          successful, filename = loop.run_until_complete(async_download_pdfs(pdf_url, filename, filenum))
          print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download {pdf_url}: HTTP {response.status_code}")

        # Be nice to the server - add some delay
        time.sleep(random.uniform(1.0, 3.0))

    except Exception as e:
        print(f"Error downloading {pdf_url}: {str(e)}")

    return successful, filename


class URL_iterator():
  #returns iteratively the next file from the URL list (as an open file). It still requires file.read() to actually read it
  def __init__(self, URL_list, output_dir):
    self.url_list = URL_list
    self.idx = 0
    self.output_dir = output_dir

  def __iter__(self):
    return self

  def __next__(self):
    if self.idx > len(self.url_list):
      raise StopIteration
    else:
      successful, filepath = download_pdfs(self.url_list[self.idx], self.output_dir, self.idx)
      self.idx += 1
      return successful, filepath



if __name__ == "__main__":
  url = "http://www.nature.com:80/polopoly_fs/1.20821!/menu/main/topColumns/topLeftColumn/pdf/538291a.pdf"
  urls = ["http://www.nature.com:80/polopoly_fs/1.20821!/menu/main/topColumns/topLeftColumn/pdf/538291a.pdf", "https://www.frontiersin.org/articles/10.3389/fpsyg.2023.1258721/pdf?isPublishedV2=False", "http://www.jaad.org/article/S0190962223009933/pdf" ]
  down_pdfs = URL_iterator(urls, "SocSciPDFs")

  pdf_filepaths = []
  for i in range(3):
    success, filepath = next(down_pdfs)
    if success:
      pdf_filepaths.append(filepath)


Downloaded: SocSciPDFs/document_0.pdf
Downloaded: SocSciPDFs/document_1.pdf
I tried the hard way, but it still failed to download http://www.jaad.org/article/S0190962223009933/pdf: HTTP Error 403
Error downloading http://www.jaad.org/article/S0190962223009933/pdf: object NoneType can't be used in 'await' expression


In [ ]:
# download_pdfs("https://www.jaad.org/article/S0190-9622(23)00993-3/pdf", "SocSciPDFs", 1)
!curl http://www.nature.com:80/polopoly_fs/1.20821!/menu/main/topColumns/topLeftColumn/pdf/538291a.pdf

url = "http://www.jaad.org/article/S0190962223009933/pdf"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/pdf,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, allow_redirects=True, headers = headers)
response.status_code

<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>openresty</center>
</body>
</html>


403

In [21]:
import glob
docs = glob.glob("SocSciPDFs/document_*")
for doc in docs:
  os.remove(doc)

In [3]:
import asyncio


In [ ]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.get("http://www.jaad.org/article/S0190962223009933/pdf")
driver = webdriver.Firefox()
driver.get("http://www.jaad.org/article/S0190962223009933/pdf")
time.sleep(5)  # Wait for authentication/redirects


WebDriverException: Message: Process unexpectedly closed with status 255


In [ ]:
!pip install geckodriver

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for geckodriver: filename=geckodriver-0.0.1-py3-none-any.whl size=1102 sha256=29c4cc0c59bf60ebe9d5155bdb0a805fbe1925f1325519c2d8e4b878bce44e44
  Stored in directory: /root/.cache/pip/wheels/7d/d4/a7/483e12ad095900063f3ee4696c29af38024675b13d66b80909
Successfully built geckodriver


In [15]:
import asyncio
from playwright.async_api import async_playwright

p = await async_playwright().start()

browser = await p.firefox.launch(headless=True)  # Set True to hide browser
page = await browser.new_page()
await page.goto("http://www.jaad.org/article/S0190962223009933/pdf")

# Wait for PDF to load
page.wait_for_timeout(3000)

# Save the PDF
pdf_content = page.pdf()
with open("article.pdf", "wb") as f:
    f.write(pdf_content)

browser.close()


Error: Page.goto: Download is starting
Call log:
  - navigating to "http://www.jaad.org/article/S0190962223009933/pdf", waiting until "load"


In [ ]:
from playwright.sync_api import Playwright, sync_playwright, expect

async with async_playwright() as p:
    #We define the browser, the context and the page
    browser = await p.firefox.launch(headless=True)
    context = await browser.new_context()
    page = await browser.new_page()
    async with page.expect_download() as download_info:
        #If we navigate without try, it will throw an exception and it will stop our script, so, we wrap it inside a try except block
        try:
            await page.goto("http://www.jaad.org/article/S0190962223009933/pdf")
        except:
            pass
    download = await download_info.value
    await download.save_as(f"./{download.suggested_filename}")
    context.close()
    browser.close()

/tmp/ipython-input-2373749230.py:16: RuntimeWarning: coroutine 'BrowserContext.close' was never awaited
  context.close()
/tmp/ipython-input-2373749230.py:17: RuntimeWarning: coroutine 'Browser.close' was never awaited
  browser.close()


In [ ]:
!playwright install-deps

Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,544 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,851 kB]
